In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Membaca data dari file CSV
df = pd.read_csv('gldm_features_Data_Enhance.csv')
print(df)
# Membagi data menjadi beberapa dataframe berdasarkan arah
directions = ['0', '45', '90', '135']
dataframes = []

for direction in directions:
    cols = [f'Contrast_{direction}', f'ASM_{direction}', f'Entropy_{direction}', f'Mean_{direction}',
            f'IDM_{direction}']
    df_direction = df[['Image'] + cols]
    df_direction['Label'] = df.index.map(lambda x: 0 if x <= 600 else 1)
    dataframes.append(df_direction)

# Fungsi untuk melatih dan menguji model Random Forest dengan preprocessing dan penyetelan hiperparameter
def train_test_RF(df):
    X = df.drop(['Image', 'Label'], axis=1)
    y = df['Label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Preprocessing data: Standarisasi
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Penyetelan hiperparameter dengan Grid Search
    param_grid = {'n_estimators': [50, 100, 200],
                  'max_depth': [None, 5, 10],
                 'max_features': ['sqrt', 'log2']}
    rf = RandomForestClassifier()
    grid_search = GridSearchCV(rf, param_grid, cv=5)
    grid_search.fit(X_train_scaled, y_train)
    best_rf = grid_search.best_estimator_

    best_rf.fit(X_train_scaled, y_train)
    y_pred = best_rf.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

# List untuk menyimpan akurasi per direction
accuracies = []

# Melakukan train dan test untuk setiap dataframe dan menambahkan akurasi ke list accuracies
for i, df_direction in enumerate(dataframes):
    accuracy = train_test_RF(df_direction)
    accuracies.append(accuracy)

# Membuat dataframe dari akurasi per direction
df_accuracy = pd.DataFrame({'Direction': directions, 'Accuracy': accuracies})

# Menghitung rata-rata akurasi dari semua direction
average_accuracy = df_accuracy['Accuracy'].mean()

# Menampilkan dataframe akurasi per direction dan rata-rata akurasi
print("Akurasi per Direction:")
print(df_accuracy)
print("\nRata-rata Akurasi: ", average_accuracy)


                      Image  Contrast_0  Contrast_45  Contrast_90   
0           Alcoholic_0.png    0.287182    -0.527308     0.375720  \
1           Alcoholic_1.png   -1.745655    -1.672462    -1.184706   
2          Alcoholic_10.png   -1.111166    -1.020774     0.129719   
3         Alcoholic_100.png   -0.632149     4.098488     2.681760   
4         Alcoholic_101.png   -0.339730     0.978283    -1.184706   
...                     ...         ...          ...          ...   
1195  Non_Alcoholic_995.png   -0.115805    -1.049446     0.115425   
1196  Non_Alcoholic_996.png   -0.894313    -0.458834    -1.184706   
1197  Non_Alcoholic_997.png   -0.258452     1.188902     1.231281   
1198  Non_Alcoholic_998.png   -1.389101     1.133665     1.203744   
1199  Non_Alcoholic_999.png    1.213954     0.673485     0.974336   

      Contrast_135     ASM_0    ASM_45    ASM_90   ASM_135  Entropy_0  ...   
0         0.328615 -0.557690 -0.557690 -0.557690 -0.557690  -0.047737  ...  \
1        -1.276

C:\Users\hanif\AppData\Local\Temp\ipykernel_22120\3676803817.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_direction['Label'] = df.index.map(lambda x: 0 if x <= 600 else 1)
C:\Users\hanif\AppData\Local\Temp\ipykernel_22120\3676803817.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_direction['Label'] = df.index.map(lambda x: 0 if x <= 600 else 1)
C:\Users\hanif\AppData\Local\Temp\ipykernel_22120\3676803817.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

Akurasi per Direction:
  Direction  Accuracy
0         0  0.695833
1        45  0.700000
2        90  0.691667
3       135  0.691667

Rata-rata Akurasi:  0.6947916666666667
